In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os
import time

print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


## 1. Set hyper parameters

In [0]:
lr = 0.001
train_epoch = 15
bsize = 100

tf.random.set_seed(777)

#### + Create checkpoint directory

In [0]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_cnn_ensemble'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

ckpt_prefix = os.path.join(checkpoint_dir, model_dir_name)

## 2. Make a data pipelining

In [4]:
mnist = keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype(np.float32) / 255.
X_train = np.expand_dims(X_train, axis=-1)
y_train = to_categorical(y_train, num_classes=10)

X_test = X_test.astype(np.float32) / 255.
X_test = np.expand_dims(X_test, axis=-1)
y_test = to_categorical(y_test, num_classes=10)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).\
                    shuffle(buffer_size=100000).batch(batch_size=bsize)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).\
                    batch(batch_size=bsize)

11493376/11490434 [==============================] - 0s 0us/step


## 3. Build a neural network model

In [0]:
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(filters=32, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool1 = keras.layers.MaxPool2D(pool_size=2, padding='same',
                                            strides=2)
        self.conv2 = keras.layers.Conv2D(filters=64, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool2 = keras.layers.MaxPool2D(pool_size=2, padding='same',
                                            strides=2)
        self.conv3 = keras.layers.Conv2D(filters=128, kernel_size=3,
                                         strides=1, padding='same',
                                         activation=tf.nn.relu)
        self.pool3 = keras.layers.MaxPool2D(pool_size=2, padding='same',
                                            strides=2)
        self.pool3_flat = keras.layers.Flatten()
        
        self.dense4 = keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.drop4 = keras.layers.Dropout(rate=0.4)
        self.dense5 = keras.layers.Dense(units=10)
        pass
    
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

models = []
num_models = 3
for i in range(num_models):
    model = MNISTModel()
    models.append(model)

## 4. Define a loss function

In [0]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean( 
                keras.losses.categorical_crossentropy(
                    y_true=labels, y_pred=logits, from_logits=True))
    return loss

## 5. Calculate a gradient

In [0]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient( loss, model.variables )

## 6. Select an optimizer

In [0]:
optimizer = keras.optimizers.Adam(learning_rate=lr)

## 7. Define a metric for model's performance

In [0]:
def evaluate(models, images, labels):
    predictions = np.zeros_like(labels)
    for model in models:
        logits = model(images, training=False)
        predictions += logits
    correct_prediction = tf.equal( x=tf.argmax(predictions, 1),
                                   y=tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))    
    return accuracy

## 8. Make a checkpoint for saving

In [0]:
checkpoints = []
for i in range(num_models):
    checkpoints.append(tf.train.Checkpoint(cnn=models[i]))

## 9. Train and Validate a neural network model

In [11]:
print('Learning started. It takes some time.')
start_time = time.time()

for epoch in range(train_epoch):
  avg_loss = 0.; avg_train_acc = 0.; avg_test_acc = 0.
  train_step = 0; test_step = 0
  
  for images, labels in train_dataset:
    for model in models:
      grads = grad(model, images, labels)
      optimizer.apply_gradients(zip(grads, model.variables))
      
      loss = loss_fn(model, images, labels)
      avg_loss += loss / num_models
    acc = evaluate(models, images, labels)
    avg_train_acc += acc / num_models
    train_step += 1
  avg_loss = avg_loss / train_step
  avg_train_acc = avg_train_acc / train_step

  for images, labels in test_dataset:
    for model in models:
      acc = evaluate(models, images, labels)
      avg_test_acc += acc
      test_step += 1
    avg_test_acc = avg_test_acc / test_step
  
  print(f'Epoch: {epoch+1:3}  |  loss = {avg_loss:10.8f}  |  ' +
        f'train_accuracy = {avg_train_acc:6.4f}  |  ' +
        f'test_accuracy = {avg_test_acc:6.4f}')

  for idx, checkpoint in enumerate(checkpoints):
    checkpoint.save(ckpt_prefix + f' - {idx}')

print('Learning Finished !')
print(f'time elapsed : {(time.time() - start_time)/60:.4f} min.')

Learning started. It takes some time.
Epoch:   1  |  loss = 0.16240056  |  train_accuracy = 0.3218  |  test_accuracy = 0.0099
Epoch:   2  |  loss = 0.03939417  |  train_accuracy = 0.3311  |  test_accuracy = 0.0100
Epoch:   3  |  loss = 0.02686219  |  train_accuracy = 0.3319  |  test_accuracy = 0.0100
Epoch:   4  |  loss = 0.01978362  |  train_accuracy = 0.3324  |  test_accuracy = 0.0100
Epoch:   5  |  loss = 0.01597871  |  train_accuracy = 0.3326  |  test_accuracy = 0.0100
Epoch:   6  |  loss = 0.01269017  |  train_accuracy = 0.3329  |  test_accuracy = 0.0100
Epoch:   7  |  loss = 0.01105772  |  train_accuracy = 0.3330  |  test_accuracy = 0.0100
Epoch:   8  |  loss = 0.00889889  |  train_accuracy = 0.3331  |  test_accuracy = 0.0100
Epoch:   9  |  loss = 0.00810749  |  train_accuracy = 0.3332  |  test_accuracy = 0.0100
Epoch:  10  |  loss = 0.00771566  |  train_accuracy = 0.3331  |  test_accuracy = 0.0100
Epoch:  11  |  loss = 0.00594616  |  train_accuracy = 0.3332  |  test_accuracy = 0